libraries that we need

In [1]:
import os
import zipfile
import pandas as pd
from collections import defaultdict
import re
import matplotlib.pyplot as plt

<h1>
extracting data from a zip file
</h1>

In [ ]:
rar_file_path = r"F:\daneshkar\project1\202405.zip" 
output_folder = 'extracted_rar_files'  

if not os.path.exists(output_folder):
    os.makedirs(output_folder)


with zipfile.ZipFile(rar_file_path, 'r') as rf:
    rf.extractall(output_folder)


for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    if filename.endswith('.zip'):
        print(f'extracting file {filename} ')
        with zipfile.ZipFile(file_path, 'r') as rf_inner:
            rf_inner.extractall(os.path.join(output_folder, filename[:-4])) 

read parquet files and create a dataframe and save them in csv files

In [ ]:
import os
import pandas as pd
import re
from collections import defaultdict

def read_and_group_parquet_files(base_folder, start_date, end_date, folders_in_row):
    data_frames = defaultdict(list)
    group_names = [
        'home_team_score',
        'home_team',
        'away_team',
        'away_team_score',
        'event',
        'round',
        'season',
        'time',
        'tournament',
        'venue',
        'odds',
        'pbp',
        'statistics',
        'power',
        'votes'
    ]

    # Regular expressions for away_team and away_team_score
    away_team_pattern = re.compile(r'^away_team_')
    away_team_score_pattern = re.compile(r'^away_team_score_')

    for date in range(start_date, end_date + 1):
        for folder in folders_in_row:
            date_folder = os.path.join(base_folder, str(date), 'data', 'raw', folder)
            print(f'Reading folder: {date_folder}')  # Monitor the current folder
            if os.path.isdir(date_folder):
                for file_name in os.listdir(date_folder):
                    if file_name.endswith('.parquet'):
                        matched_group = None
                        # Check each group name if the filename starts with it
                        if away_team_score_pattern.match(file_name):
                            matched_group = 'away_team_score'
                        elif away_team_pattern.match(file_name):
                            matched_group = 'away_team'
                        else:
                            # Simple string matching for other group names
                            for group_name in group_names:
                                if file_name.startswith(group_name + '_'):
                                    matched_group = group_name
                                    break

                        if matched_group:
                            file_path = os.path.join(date_folder, file_name)
                            df = pd.read_parquet(file_path)
                            data_frames[matched_group].append(df)

    # Concatenate DataFrames in each group and save to CSV
    grouped_data_frames = {}
    for group, dfs in data_frames.items():
        if dfs:  # Check if the list of DataFrames is not empty
            concatenated_df = pd.concat(dfs, ignore_index=True)
            grouped_data_frames[group] = concatenated_df
            # Save DataFrame to CSV
            csv_file_path = os.path.join(base_folder, f'{group}.csv')
            concatenated_df.to_csv(csv_file_path, index=False)
            print(f'Saved {group} DataFrame to {csv_file_path}')

    return grouped_data_frames

# Example usage
base_folder = r'F:\daneshkar\project1\extracted_rar_files'
start_date = 20240501
end_date = 20240531
folders_in_row = (
    'raw_match_parquet',
    'raw_odds_parquet',
    'raw_point_by_point_parquet',
    'raw_statistics_parquet',
    'raw_tennis_power_parquet',
    'raw_votes_parquet'
)

grouped_data_frames = read_and_group_parquet_files(base_folder, start_date, end_date, folders_in_row)

# Display the groups and their DataFrames
for group_name, df in grouped_data_frames.items():
    print(f'Group: {group_name}')
    # print(df.head())  # Print the first few rows of each DataFrame


read csv files

In [ ]:
# Define the directory and group names
base_folder = r'F:\daneshkar\project1\extracted_rar_files'
group_names = [
    'home_team_score',
    'home_team',
    'away_team',
    'away_team_score',
    'event',
    'round',
    'season',
    'time',
    'tournament',
    'venue',
    'odds',
    'pbp',
    'statistics',
    'power',
    'votes'
]

# Initialize a dictionary to store the DataFrames
grouped_data_frames = {}

# Read each CSV file into a DataFrame and store it in the dictionary
for group_name in group_names:
    csv_file_path = os.path.join(base_folder, f'{group_name}.csv')
    if os.path.exists(csv_file_path):
        df = pd.read_csv(csv_file_path)
        grouped_data_frames[group_name] = df
        print(f'Successfully read {group_name}.csv')
    else:
        print(f'File {group_name}.csv not found')

# Display the DataFrames (or perform further operations)
for group_name, df in grouped_data_frames.items():
    print(f'Group: {group_name}')
    # print(df.head())  # Print the first few rows of each DataFrame


check the shape of dataframes

In [ ]:
# Display the shape of each DataFrame
for group_name, df in grouped_data_frames.items():
    print(f'Group: {group_name}, Shape: {df.shape}')

1. How many tennis players are included in the dataset?

In [ ]:
def count_unique_players(grouped_data_frames):
    player_ids = set()
    
    # Extract player IDs from home team info
    if 'home_team' in grouped_data_frames:
        home_team_df = grouped_data_frames['home_team']
        if 'player_id' in home_team_df.columns:
            player_ids.update(home_team_df['player_id'].unique())
    
    # Extract player IDs from away team info
    if 'away_team' in grouped_data_frames:
        away_team_df = grouped_data_frames['away_team']
        if 'player_id' in away_team_df.columns:
            player_ids.update(away_team_df['player_id'].unique())
    
    # Return the count of unique player IDs
    return len(player_ids)

# Count unique players
num_unique_players = count_unique_players(grouped_data_frames)
print(f'The number of unique tennis players in the dataset is: {num_unique_players}')


otherway

In [ ]:
event = grouped_data_frames['event']
home_team = grouped_data_frames['home_team']
away_team = grouped_data_frames['away_team']

# Combine home and away team info into one DataFrame
players_info = pd.concat([
    home_team[['player_id', 'full_name']],
    away_team[['player_id', 'full_name']]
]).drop_duplicates().reset_index(drop=True)
number_of_tennis_players=len(players_info)

print(f'The number of unique tennis players in the dataset is: {number_of_tennis_players}')

<h1>
2.What is the average height of the players?
</h1>

In [ ]:
df_cleaned_away_team = grouped_data_frames['away_team'].dropna(subset=['gender'])
df_cleaned_home_team = grouped_data_frames['home_team'].dropna(subset=['gender'])
#We remove the rows that have missing values in the gender column.

df_concat_gender=pd.concat([df_cleaned_home_team[['player_id','full_name','gender','height']],df_cleaned_away_team[['player_id','full_name','gender','height']]]).drop_duplicates().reset_index(drop=True)
mean_height_male_with_null = df_concat_gender[df_concat_gender['gender'] == 'M']['height'].mean()
mean_height_female_with_null = df_concat_gender[df_concat_gender['gender'] == 'F']['height'].mean()
num_missing = df_concat_gender['gender'].isna().sum()
def fill_height(row):
    if pd.isna(row['height']):
        if row['gender'] == 'M':
            return mean_height_male_with_null
        elif row['gender'] == 'F':
            return mean_height_female_with_null
    return row['height']
df_concat_gender['height'] = df_concat_gender.apply(fill_height, axis=1)
mean_height_male = df_concat_gender[df_concat_gender['gender'] == 'M']['height'].mean()
mean_height_female = df_concat_gender[df_concat_gender['gender'] == 'F']['height'].mean()
mean_height_players=df_concat_gender['height'].mean()
print(f'The average height of female players is {mean_height_female} ')
print(f'The average height of male players is {mean_height_male}')
print(f'The average height of players is {mean_height_players}')

3.Which player has the highest number of wins?

In [ ]:
import pandas as pd

# Assuming grouped_data_frames is your dictionary containing all the dataframes
event = grouped_data_frames['event']
home_team = grouped_data_frames['home_team']
away_team = grouped_data_frames['away_team']

# Combine home and away team info into one DataFrame
players_info = pd.concat([
    home_team[['player_id', 'full_name']],
    away_team[['player_id', 'full_name']]
]).drop_duplicates().reset_index(drop=True)

# Merge event DataFrame with home_team and away_team DataFrames to get player IDs
event = event.merge(home_team[['match_id', 'player_id']], left_on='match_id', right_on='match_id', how='left', suffixes=('', '_home'))
event = event.rename(columns={'player_id': 'player_id_home'})  # Rename the merged column from home_team
event = event.merge(away_team[['match_id', 'player_id']], left_on='match_id', right_on='match_id', how='left', suffixes=('', '_away'))
event = event.rename(columns={'player_id': 'player_id_away'})  # Rename the merged column from away_team



# Map winner_code to actual player IDs
def get_winner_player_id(row):
    if row['winner_code'] == 1:
        return row['player_id_home']
    elif row['winner_code'] == 2:
        return row['player_id_away']
    else:
        return None

# Check if 'player_id_home' and 'player_id_away' columns are present
if 'player_id_home' in event.columns and 'player_id_away' in event.columns:
    event['winner_player_id'] = event.apply(get_winner_player_id, axis=1)
else:
    raise KeyError("Expected columns 'player_id_home' and 'player_id_away' not found in the event DataFrame.")

# Count the number of wins for each player using groupby
win_counts = event.groupby('winner_player_id').size().reset_index(name='win_count').rename(columns={'winner_player_id': 'player_id'})

print("Win Counts DataFrame:")
print(win_counts)

# Merge win counts with player information
player_wins = pd.merge(win_counts, players_info, on='player_id', how='left')

print("Player Wins DataFrame:")
print(player_wins)

# Find the player with the highest number of wins
if not player_wins.empty:
    top_player = player_wins.loc[player_wins['win_count'].idxmax()]
    print(f"Player with the highest number of wins: {top_player['full_name']} with {top_player['win_count']} wins")
else:
    print("No wins found.")


<h1>
4.What is the longest match recorded in terms of duration?
</h1>

In [ ]:
df_time=grouped_data_frames['time']
df_time[df_time['current_period_start_timestamp'].isna()]
df_time['datetime'] = pd.to_datetime(df_time['current_period_start_timestamp'], unit='s')
columns_to_check = df_time.columns[1:6]
df_time_filtered = df_time.dropna(subset=columns_to_check, how='all')
match_recorded_column_data = df_time.iloc[:, 1:6].sum(axis=1)


insert match_recorded to table of time


In [ ]:
df_time=df_time.insert(loc=6, column='Match recorded', value=match_recorded_column_data)

write condition

In [ ]:
condition = match_recorded_column_data <= 39600
#The longest tennis set lasted 11 hours, so we don't consider data beyond this timeframe.
if condition.any():
   
    max_row = df_time.loc[condition].iloc[:, 0:5].sum(axis=1).idxmax()
    df_time = df_time.drop(columns=df_time.columns[1:6])
    result_row = df_time.loc[max_row]

print(result_row)

5.How many sets are typically played in a tennis match?

In [ ]:
home_team_score_df = grouped_data_frames['home_team_score']
# Select columns period_1 to period_5
period_columns = ['period_1', 'period_2', 'period_3', 'period_4', 'period_5']

# Count non-null values across rows (axis=1)
home_team_score_df['filled_columns_count'] = home_team_score_df[period_columns].count(axis=1)

# Find the mode of filled_sets_count
mode_sets_count = home_team_score_df['filled_columns_count'].mode().values[0]

print(f"Typically, {mode_sets_count} sets are played in a tennis match.")

<h1>
6.Which country has produced the most successful tennis players?
</h1>

In [ ]:
df_all_rank=pd.concat([grouped_data_frames['home_team'],grouped_data_frames['away_team']])
df_rank = df_all_rank[(df_all_rank['current_rank'] <= 20) ]
df_rank_w=df_rank.drop(columns=['match_id','user_count','current_prize','total_prize'])
df_rank_duplicates=df_rank_w.drop_duplicates()
sorted_df = df_rank_duplicates.sort_values(by='current_rank', ascending=True)

df_unique_rank =  sorted_df.groupby('name').first().reset_index()

value_counts=df_unique_rank['country'].value_counts()
value_counts.plot(kind='bar',color='green')
plt.yticks(range(int(value_counts.max()) + 1))
plt.show()
max_index = value_counts.idxmax()
countries_with_max_value = value_counts[value_counts == value_counts[max_index]].index.tolist()

print(f"{', '.join(map(str, countries_with_max_value[:-1]))} , {countries_with_max_value[-1]}")

7.What is the average number of aces per match?

first we check the symmetry of data by the histogram of data to chose between mean and median.

In [ ]:

statistics_df = grouped_data_frames['statistics']

# Filter statistics dataframe for rows where statistic_name is 'aces' and period is 'ALL'
aces_df = statistics_df[(statistics_df['statistic_name'] == 'aces') & (statistics_df['period'] == 'ALL')].copy()

# Convert 'home_stat' and 'away_stat' to numeric if they are not already
aces_df.loc[:, 'home_stat'] = pd.to_numeric(aces_df['home_stat'], errors='coerce')
aces_df.loc[:, 'away_stat'] = pd.to_numeric(aces_df['away_stat'], errors='coerce')

# Calculate the total_stat as the sum of home_stat and away_stat
aces_df['total_stat'] = aces_df['home_stat'] + aces_df['away_stat']

# Plot histogram to check distribution
plt.figure(figsize=(10, 6))
plt.hist(aces_df['total_stat'].dropna(), bins=30, edgecolor='k', alpha=0.7)
plt.title('Histogram of Total Stat for "aces" and Period="ALL"', fontsize=14)
plt.xlabel('Total Stat', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True)
plt.show()

# Calculate mean and median
mean_total_stat = aces_df['total_stat'].mean()
median_total_stat = aces_df['total_stat'].median()

# Print mean and median
print(f"Mean of total_stat: {mean_total_stat:.2f}")
print(f"Median of total_stat: {median_total_stat:.2f}")


boxplot can help identify skewness and outliers

In [ ]:
plt.figure(figsize=(8, 6))
plt.boxplot(aces_df['total_stat'], vert=False, patch_artist=True, widths=0.7)
plt.title('Boxplot of total_stat for "aces" and period="ALL"', fontsize=14)
plt.xlabel('Total Stat', fontsize=12)
plt.yticks([], [])
plt.grid(True)
plt.show()

calculate the average number of aces per match

In [ ]:
# Calculate the median of total_stat
median_total_stat = aces_df['total_stat'].median()

# Print the mean of total_stat
print(f"the average number of aces per match is: {median_total_stat:.2f}")


the matches that have more that 13 ace

In [ ]:
aces_df[['match_id','total_stat']].loc[aces_df['total_stat']>13].count()

<h1>
8.Is there a difference in the number of double faults based on gender?
</h1>

In [ ]:
df_statistics=grouped_data_frames['statistics']

df_away_team=grouped_data_frames['away_team'][['match_id','full_name','gender']]
df_home_team=grouped_data_frames['home_team'][['match_id','full_name','gender']]

df_concat_team=pd.concat([df_away_team,df_home_team])

filtered_df_statistics = df_statistics[df_statistics['statistic_name'] =='double_faults' ][['match_id']]
grouped_filtered_df_statistics = filtered_df_statistics.groupby('match_id').size().reset_index(name='Count')
merged_df = pd.merge(grouped_filtered_df_statistics, df_concat_team, on='match_id', how='inner')
merged_df_deduplicated_by_columns = merged_df.drop_duplicates(subset=['match_id']).reset_index(drop=True)
gender_count_sum = merged_df_deduplicated_by_columns.groupby('gender')['Count'].sum().reset_index()

9.Which player has won the most tournaments in a single month?

In [ ]:
import pandas as pd

# Assuming grouped_data_frames is your dictionary containing all the dataframes
event_df = grouped_data_frames['event']
home_team_df = grouped_data_frames['home_team']
away_team_df = grouped_data_frames['away_team']
tournament_df = grouped_data_frames['tournament']

# Merge dataframes to get the necessary information
merged_df = event_df.merge(home_team_df[['match_id', 'player_id', 'name']], on='match_id', suffixes=('_home', '_away'))
merged_df = merged_df.merge(away_team_df[['match_id', 'player_id']], on='match_id', suffixes=('', '_away'))
merged_df = merged_df.merge(tournament_df[['match_id', 'tournament_id', 'tournament_name']], on='match_id')

# Determine the winner's player_id for each match
merged_df['winner_player_id'] = merged_df.apply(
    lambda row: row['player_id'] if row['winner_code'] == 1 else row['player_id_away'],
    axis=1
)

# Count the number of tournaments won by each player
tournament_wins = merged_df.groupby('winner_player_id').size().reset_index(name='tournament_wins')

# Find the player with the maximum number of tournament wins
max_wins = tournament_wins.loc[tournament_wins['tournament_wins'].idxmax()]

# Get the player's name
player_info = home_team_df[['player_id', 'name']].drop_duplicates().set_index('player_id')
max_wins_player_name = player_info.loc[max_wins['winner_player_id'], 'name']
max_wins_count = max_wins['tournament_wins']

print(f"Player {max_wins_player_name} won the most tournaments ({max_wins_count}) in the given month.")


<h1>
10.Is there a correlation between a player's height and their ranking?
</h1>

In [ ]:
df_concat_team=pd.concat([grouped_data_frames['away_team'],grouped_data_frames['home_team']])
df_concat_team=df_concat_team.drop(columns=['match_id','user_count','current_prize','total_prize'])
df_concat_team=df_concat_team.drop_duplicates()
df_concat_team=df_concat_team.groupby('name').first().reset_index()
correlation = df_concat_team['height'].corr(df_concat_team['current_rank'])
if correlation >= 0.7 or correlation <= -0.7:
    correlation_category = "Strong correlation"
elif 0.4 <= correlation < 0.7 or -0.4 >= correlation > -0.7:
    correlation_category = "Moderate correlation"
elif 0.1 <= correlation < 0.4 or -0.1 >= correlation > -0.4:
    correlation_category = "Weak correlation"
else:
    correlation_category = "Very weak or no correlation"


print(f"The correlation between Column1 and Column2 is: {correlation}")
print(f"This correlation is categorized as: {correlation_category}")

11.What is the average duration of matches?

In [ ]:

time_df = grouped_data_frames['time']

# Ensure all period columns are treated as numeric, and NaNs are handled
time_df = time_df[['match_id', 'period_1', 'period_2', 'period_3', 'period_4', 'period_5']].apply(pd.to_numeric, errors='coerce')

# Calculate the total duration for each match by summing the periods
time_df['total_duration'] = time_df[['period_1', 'period_2', 'period_3', 'period_4', 'period_5']].sum(axis=1, skipna=True)

# Calculate the average duration
average_duration = time_df['total_duration'].mean()

print(f"The average duration of matches is {average_duration/60} minutes.")



<h1>
12-What is the average number of games per set in men's matches compared to women's matches?
</h1>

In [ ]:

nan_count = grouped_data_frames['power']['set_num'].isna().sum()
#check for null value
df_team_without_filter=pd.concat([grouped_data_frames['away_team'],grouped_data_frames['home_team']])[['gender','match_id']]
condition = df_team_without_filter['gender'].notna()
df_team_with_filter=df_team_without_filter[condition]
df_merge_team_power=pd.merge(df_team_with_filter,grouped_data_frames['power'][['match_id','set_num','game_num']] , on='match_id', how='inner')
count_set=len(df_merge_team_power['set_num'].unique())
grouped = df_merge_team_power.groupby('gender')['game_num'].sum()

result = grouped / count_set

13.What is the distribution of left-handed versus right-handed players?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming grouped_data_frames is your dictionary containing all the dataframes
home_team_df = grouped_data_frames['home_team']
away_team_df = grouped_data_frames['away_team']

# Combine the relevant data to get a comprehensive list of players
combined_players_df = pd.concat([home_team_df[['player_id', 'plays']], away_team_df[['player_id', 'plays']]])

# Drop duplicate players if they appear in both home_team and away_team
players_df = combined_players_df.drop_duplicates(subset=['player_id']).copy()

# Check the unique values in the 'plays' column
unique_plays = players_df['plays'].unique()
print("Unique values in 'plays' column:")
print(unique_plays)

# Check for missing or NaN values
missing_plays = players_df['plays'].isna().sum()
print(f"Number of missing or NaN values in 'plays' column: {missing_plays}")

# Fill or handle missing values (assuming 'unknown' for missing entries)
players_df['plays'] = players_df['plays'].fillna('unknown')

# Calculate the distribution of left-handed vs right-handed players
handedness_distribution = players_df['plays'].value_counts()

# Print the distribution
print("Distribution of left-handed vs right-handed players:")
print(handedness_distribution)

# Plot the distribution using a pie chart
explode = [0.1 if handedness == "ambidextrous" else 0 for handedness in handedness_distribution.index]
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']  # Optional: Define custom colors

handedness_distribution.plot(kind='pie', autopct='%1.1f%%', startangle=90, figsize=(8, 8), legend=True, explode=explode, colors=colors)
plt.title('Distribution of Left-handed vs Right-handed Players')
plt.ylabel('')  # Hide the y-label
plt.legend(title='Handedness')
plt.show()

<h1>
14.What is the most common type of surface used in tournaments?
</h1>

In [ ]:
grouped_data_frames['tournament']['ground_type'].value_counts().idxmax()

15.How many distinct countries are represented in the dataset?

In [ ]:

home_team_df = grouped_data_frames['home_team']
away_team_df = grouped_data_frames['away_team']

# Combine the country data from both home_team and away_team dataframes
countries = pd.concat([home_team_df['country'], away_team_df['country']])

# Find the number of distinct countries
distinct_countries = countries.nunique()

print(f"Number of distinct countries represented in the dataset: {distinct_countries}")


<h1>
16.What is the most common type of surface used in tournaments?
</h1>

In [ ]:
dffi=grouped_data_frames['pbp']
last_row_per_id = dffi.groupby('match_id').last().reset_index()
def determine_winner(row):
    home_has_A = 'A' in str(row['home_point'])
    away_has_A = 'A' in str(row['away_point'])

    if home_has_A and not away_has_A:
        return 0  
    elif away_has_A and not home_has_A:
        return 1  
    elif '40' in [row['home_point'], row['away_point']]:
        return 0 if row['home_point'] == '40' else 1  
    else:
        return None


last_row_per_id['Winner'] = last_row_per_id.apply(determine_winner, axis=1)
df_home_new=grouped_data_frames['home_team']
df_home_new['h/a']=0
df_away_new=grouped_data_frames['away_team']
df_away_new['h/a']=1
df_home_away=pd.concat([df_home_new,df_away_new])
df_merge_team_pbp=pd.merge(df_home_away,last_row_per_id[['Winner','match_id']] ,on='match_id',how='inner')
df_merge_team_pbp=df_merge_team_pbp[['match_id','name', 'gender','current_rank','h/a','Winner']].sort_values(by='match_id')
grouped = df_merge_team_pbp.groupby('match_id').filter(lambda x: (x['current_rank'] <= 10).any()).reset_index(drop=True)
filterede_df = grouped[(grouped['h/a'] == grouped['Winner']) & (grouped['current_rank'] > 10)]
count=grouped[(grouped['current_rank'] > 10)].reset_index(drop=True)


count_df1 = filterede_df['name'].value_counts().reset_index()
count_df1.columns = ['name', 'wincounts']

count_df2 = count['name'].value_counts().reset_index()
count_df2.columns=['name','matchcount']


df_count_all=pd.merge(count_df1,count_df2,on='name',how='inner').reset_index(drop=True)
df_count_all['win percentage']=df_count_all['wincounts']*100/df_count_all['matchcount']
df_count_all=df_count_all.sort_values(by='win percentage',ascending=False)
df_count_all[df_count_all['win percentage']==100].reset_index(drop=True)

17.What is the average number of breaks of serve per match?

In [ ]:
import pandas as pd

# Assuming grouped_data_frames is your dictionary containing all the dataframes
home_team_score_df = grouped_data_frames['home_team_score']

# Select columns period_1_tie_break to period_5_tie_break
period_tie_break_columns = ['period_1_tie_break', 'period_2_tie_break', 'period_3_tie_break', 'period_4_tie_break', 'period_5_tie_break']

# Count non-null values across rows (axis=1)
home_team_score_df['filled_columns_count'] = home_team_score_df[period_tie_break_columns].count(axis=1)

# Find the median of filled_columns_count
median_sets_count = home_team_score_df['filled_columns_count'].median()

print(f"Typically, the median number of sets played in a tennis match is {median_sets_count}.")


to see better :

In [ ]:
import matplotlib.pyplot as plt

# Create a bar plot for the number of break in tennis matches
sets_count = home_team_score_df['filled_columns_count'].value_counts().sort_index()

plt.figure(figsize=(10, 5))
sets_count.plot(kind='bar')
plt.title('Bar Plot of Number of breaks in Tennis Matches')
plt.xlabel('Number of Sets')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.show()
